In [1]:
import logging
import os

for log_file in [ "logger.log", "error.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Log file {log_file} are cleaned.")

logging.basicConfig(
    filename='logger.log',
    level=logging.DEBUG,
    format= '%(filename)s:%(lineno)s %(levelname)s: %(message)s'
)

print("Logging configured.")


Log file logger.log are cleaned.
Logging configured.


In [34]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search
from google.genai import types
from typing import List
from google.adk.runners import InMemoryRunner


In [36]:
retry_config = types.HttpRetryOptionsDict(
    attempts=5,
    exp_base= 7,
    initial_delay= 1,
    http_status_codes=[429,500,503,504]
)

In [37]:
def paper_count(papers: str):
    """  This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list."""
    
    return len(papers)

In [38]:
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options= retry_config),
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)

In [39]:
root_agent = LlmAgent(
    name= "root_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options= retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.""",
    tools=[paper_count, AgentTool(agent=google_search_agent)]
)

In [41]:
runner = InMemoryRunner(agent= root_agent)

In [42]:
user_message = "what are the research on alopecia areta?"
response = runner.run_async(
    user_id='default',
    session_id='default_session',
    new_message=user_message
)

print(response)

<async_generator object Runner.run_async at 0x0000022B3ED1EAC0>
